<center> <img src = https://play-lh.googleusercontent.com/YpAV7Q-ZJhI5tzFk_wEX-7-x2BydtnCtFTVUrmq0zAO6jLCLA4nNcfem3p_Pyowg9w width='350'>

# <center> Проект SQL: Анализ вакансий из HeadHunter  

In [4]:
import psycopg2 # библиотека для работы phyton с СУБД PostgreSQL
import pandas as pd
# Импортируем библиотеку plotly для визуализации результата
import plotly.express as px

## Предварительный анализ данных

#### Количество вакансий

In [10]:
# Используем БД - project_sql, схему - public
connection_project = psycopg2.connect(
   dbname='project_sql',
   user='skillfactory',
   host='84.201.134.129',
   password='cCkxxLVrDE8EbvjueeMedPKt',
   port=5432
   )

query_3_1 = f''' 
            select count(*)
            from public.vacancies
            limit 10
            '''
            # считаем кол-во вакансий

In [11]:
exit_3_1 = pd.read_sql_query(query_3_1, connection_project)# делаем запрос к таблице vacancies
display(exit_3_1)

,count
0,49197


#### Количество работодателей

In [12]:
query_3_2 = f''' 
            select COUNT(*)
            FROM public.employers 
            '''

In [13]:
exit_3_2 = pd.read_sql_query(query_3_2, connection_project)
display(exit_3_2)

,count
0,23501


Для выводов по количеству вакансий и работодателей проведём дополнительное исследование, что бы оценить распределение вакантных мест среди компаний

#### **Дополнительное исследование №1**

In [14]:
# Построим график распределения кол-ва вакансий среди работодателей 
query_test = f'''
    select 
        employers.name,
        count(vacancies.id)
    from public.employers
    join public.vacancies ON vacancies.employer_id = employers.id
    group by employers.id
            '''
plot = pd.read_sql_query(query_test, connection_project)

# Исключим работодателя "Яндекс" для более точной оценки менее крупных компаний
plot_Y = plot[plot['name'] != 'Яндекс']
scat_plot = px.scatter(data_frame = plot_Y,
       x= 'name',
       y= 'count'
          )
scat_plot.show()
scat_plot.write_html("plotly_express1.html")

Количество **вакансий** (49197) примерно вдвое больше количества **работодателей** (23501).
Это может говорить о конкуренции в рынке и о темпе развития молодых компаний.
Например, в ситуации с малым количеством работодателей, можно было бы предположить наличие множества монополий на продукцию. Поскольку значительную массу вакансий разместят крупные фирмы.
На графике же видно, что основная масса вакансий распределена равномерно и находится на уровне от 1 до 30 вакансий - значит на рынке множество молодых и развивающихся компаний

#### Количество регионов

In [15]:
query_3_3 = f''' 
            SELECT COUNT(*)
            FROM public.areas
            '''

In [16]:
exit_3_3 = pd.read_sql_query(query_3_3, connection_project)
display(exit_3_3)

,count
0,1362


Мы имеем большое количество регионов, в которых может быть неравномерное распределение вакансий и регионов регистрации работодателей. При такой большой территории, вероятно, есть регионы, не имеющие данных. Необходимо узнать распределение вакансйи по территории.

#### Количество сфер деятельности

In [17]:
query_3_4 = f''' 
            SELECT COUNT(*)
            FROM public.industries
            '''

In [18]:
exit_3_4 = pd.read_sql_query(query_3_4, connection_project)
display(exit_3_4)

,count
0,294


Посмотрим распределение сфер деятельности. Для этого проведем дополнительное исследование

#### **Дополнительное исследование №2**

In [19]:
# Создадим запрос, который считает кол-во сфер деятельности, указанных работодателями
query_test3 = f''' 
            SELECT
                ind.name,
                COUNT(ind.name) 
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                JOIN public.employers em ON em_i.employer_id = em.id
            GROUP BY ind.name
            '''
exit_test3 = pd.read_sql_query(query_test3, connection_project)

# Построим график с отрослями, указанных работодателями более 700 раз
plot_ind = px.bar(data_frame= exit_test3[exit_test3['count'] > 300],
                  x= 'name',
                  y= 'count')
plot_ind.show()
plot_ind.write_html("plotly_express2.html")

По этому графику можно вынести сразу несколько предположений.
Рынок активно развивается в сфере компьютерных технологий, в особенности переключение всего бизнеса в цифровую среду.
<span style="color:#59afe1">Отрасль, связанная с интернет-рекламой (Интернет-компания), является уже устоявшейся.</span> - практически каждый бизнес делает продвижение через соц-сети, рекламу в поисковиках и прочую интернет-рекламу. Хоть таких работодателей среди наиболее популярных отраслей меньшенство, сама отрасль входит в одну из наиболее востребованных. Поиск работы в этой сфере будет тяжелее,чем программисту, поскольку компании уже имеют специалистов этой сферы и спрос будет ниже. 

На первом месте отрасль по разработке програмного обеспечения. Бизнес становится эффективнее благодаря оптимизации многих процессов - сбор информации о клиентах, проведение расчетов, учет продукции и др.
По графику можно сказать, что рынок только начал развитие в этой сфере деятельности,значит спрос на соискателей, владеющих языками программирования, высокий и хорошо оплачиваемый. <span style="color:#59afe1"> Это хорошие возможности для начинающих специалистов в этой области, различных школ и курсов по обучению.</span>

# Детальный анализ вакансий

#### Количество вакансий в каждом регионе. Выведм пятёрку лидеров по количеству вакансий

In [20]:
query_4_1 = f''' 
            SELECT
                ar.name AS area,
                COUNT(*) AS cnt
            FROM 
                public.vacancies AS vc -- Для получения названий регионов и городов соединяем таблицы vacancies и areas
                JOIN public.areas AS ar ON vc.area_id = ar.id
            GROUP BY ar.name
            ORDER BY COUNT(*) DESC 
            LIMIT 5
            '''

In [21]:
exit_4_1 = pd.read_sql_query(query_4_1, connection_project)
display(exit_4_1)

plot = px.bar(data_frame = exit_4_1,
              x = 'cnt',
              height = 400,
              width = 700,
              color = 'area',
              orientation = 'h'
              )
plot.show()
plot.write_html("plotly_express3.html")

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


Можно сделать вывод, что в этих городах высокие спрос и предложение на работу и,следовательно, заработная плата выше в сравнении с другими городами, поскольку создается хорошая конкуренция среди компаний и работников.
Москва является самым концентрированным городом по вакансиям - это связано с быстрым развитием города, широким спектром разных доходов людей  и тп, что способствует быстрому развитию бизнеса.

#### Количество вакансий с заполненым хотя бы одним из двух полей с зарплатой

In [ ]:
query_4_2 = f'''
            SELECT 
                COUNT(*)
            FROM public.vacancies vc
            WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL 
            
            '''

In [ ]:
exit_4_2 = pd.read_sql_query(query_4_2, connection_project)
display(exit_4_2)

,count
0,24073


Примерно в половине всех вакансий не указа зарплатная вилка. Это может говорить о неуверенности работодателей в том, какой именно специалист ему необходим для выполнения тех или иных задач, какие компетенции будет выполнять сотрудник и какую пользу принесет его работа.
Так же можно предположить, что компании ищут кандидатов, знающих цену своей работе и ориентирующихся в рынке.

#### Cредние значения для нижней и верхней границы зарплатной вилки.

In [ ]:
query_4_3 = f''' 
            SELECT
                AVG(salary_from) AS lower_mean,
                AVG(salary_to) AS upper_mean
            FROM public.vacancies
            '''

In [ ]:
exit_4_3 = pd.read_sql_query(query_4_3, connection_project)
display(exit_4_3)

,lower_mean,upper_mean
0,71064.657901,110536.741923


#### Количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.

In [ ]:
query_4_4 = f''' 
            SELECT
                schedule,
                employment,
                COUNT(*)
            FROM public.vacancies
            GROUP BY schedule,employment
            ORDER BY COUNT(*) DESC
            limit 5
            '''

In [ ]:
exit_4_4 = pd.read_sql_query(query_4_4, connection_project)
display(exit_4_4)

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


Заметное большинство вакансий указывается с графиком Полный день и трудоустройством Полная занятость, такой вариант характерен для работы в офисе. Можно предположить, что значительное большинство компаний хотят иметь возможность контролировать работу своих сотрудников, также помогать и направлять их эффективнее оффлайн. Сочетание этих условий попогает добиться хорошей отдачи от работы в виде результата и выстраиванию долгосрочного сотрудничества с работником.

#### Выведем значения поля experience в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [52]:
query_4_5 = f''' 
            SELECT
                experience,
                COUNT(*)
            FROM public.vacancies
            GROUP BY experience
            ORDER BY COUNT(*)
            '''

In [53]:
exit_4_5 = pd.read_sql_query(query_4_5, connection_project)
display(exit_4_5)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Наиболее востребованы специалисты начинающие и с опытом от 3 до 6 лет. Молодые и начинающие специалисты развиваются в компании и становятся опытными специалистами глубоко понимающими задачи организации. Таких сотрудников нанимают в основном устойчивые на рынке компании, имеющие возможность "растить" новых специалистов и расширяться. Можно предположить что мы имеем дело с разивающимся рынком.
Молдые компании, пока не готовые обучать малоопытных специалистов, ищут сотруднка со средним или большим опытом, что бы быстро получить результат его работы и наладить выполнение недостающей для бизнеса части работы.

# Анализ работодателей

#### Какие работодатели находятся на первом и пятом месте по количеству вакансий

In [ ]:
query_5_1 = f''' 
            SELECT 
                empl.name,
                COUNT(*)
            FROM public.employers empl
                JOIN public.vacancies vc ON vc.employer_id = empl.id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 5
            '''

In [ ]:
exit_5_1 = pd.read_sql_query(query_5_1, connection_project)
display(exit_5_1)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


Из вывода следует что Яндекс разместил вакансий больше остальных. Следовательно крупная компания нацелена на расширение и развитие бизнеса в разных секторах. Скорее всего Яндекс, в отличии от других компаний, имеет больше представлений разной продукции на рынке, что делает его бренд узнаваемым (в том числе и компаний вошедших в топ-5). Предположу, что Яндекс расширяет бизнес за счет человеческих ресурсов для предоставления услуг, доступных конечному потребителю (физ.лицам).

#### Для каждого региона выведем количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдем тот, в котором наибольшее количество работодателей

In [5]:
query_5_2 = f''' 
            SELECT
                ar.name,
                COUNT(DISTINCT vc.id) vc,
                COUNT(DISTINCT em.id) emp
            FROM public.areas ar
                LEFT JOIN public.vacancies vc ON ar.id = vc.area_id
                LEFT JOIN public.employers em ON ar.id = em.area
            GROUP BY 1
            ORDER BY vc ASC, emp DESC
            '''

In [6]:
exit_5_2 = pd.read_sql_query(query_5_2, connection_project)
display(exit_5_2)

,name,vc,emp
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
1357,Алматы,1892,721
1358,Новосибирск,2006,573
1359,Минск,2112,1115
1360,Санкт-Петербург,2851,2217


Регион регистрации некоторых работодателей указывается как страна, а не город. Отсутствующие вакансии в регионах не всегда означают, что их там нет (Посокльку регион Россия имеет вакансии). Причина может быть невозможность указать в вакансии регион как Страну, а не город.

#### Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

In [ ]:
query_5_3 = f''' 
            SELECT 
                em.name,
                COUNT(DISTINCT vc.area_id)
            FROM public.employers em
                JOIN public.vacancies vc ON em.id = vc.employer_id
            GROUP BY 1
            ORDER BY 2 desc
            '''

In [ ]:
exit_5_3 = pd.read_sql_query(query_5_3, connection_project)
display(exit_5_3)

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


Компании с боьшим числом регионов, в которых размещены вакансии, иеют бльшое покрытие по стране и делают свой продукт популярнее. Такой разброс может означать, что компании имеют бизнес в сфере обслуживания. Поэтому для привлечения новых сотрудников подходят многие регионы имеющие потенциал использования услуг населением.

#### Количество работодателей, у которых не указана сфера деятельности.

In [ ]:
query_5_4 = f''' 
            SELECT
                COUNT(*)
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                RIGHT JOIN public.employers em ON em_i.employer_id = em.id
            WHERE ind.name IS NULL
            '''

In [ ]:
exit_5_4 = pd.read_sql_query(query_5_4, connection_project)
display(exit_5_4)

,count
0,8419


Более 8 тысяч работодателей не указали сферу деятельности. Возможно эту информацию просто не указали или это связано с определенной секретностью. Поскольку понять, чем занимается организация нельзя напрямую, можно проанализировать наличие вакансий у таких работодателей и установить род деятельности. Если у компании отсутствуют и вакансии, то такая запись в данных не принесет пользы.

#### Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [ ]:
query_5_5 = f''' 
            SELECT
                em.name,
                COUNT(industry_id) cnt_industry
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                JOIN public.employers em ON em_i.employer_id = em.id
            GROUP BY em.name
            HAVING COUNT(industry_id) = 4
            ORDER BY em.name
            '''

In [ ]:
exit_5_5 = pd.read_sql_query(query_5_5,connection_project)
display(exit_5_5)

,name,cnt_industry
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


Работодателей с четырьмя сферами деятельности более 1к, такие компании могут быть слабоизвестны если каждый из бизнесов, который они ведут, не имеет широкого распространения своего продукта. Также организация может работать в сферах B2B и конечные потребители могут не знать о ней.

#### Количество работодателей, у которых сфера деятельности указана «Разработка программного обеспечения».

In [ ]:
query_5_6 = f''' 
            SELECT
                COUNT(em.id)
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                JOIN public.employers em ON em_i.employer_id = em.id
            WHERE ind.name LIKE '%Разработка программного обеспечения%'
            '''

In [ ]:
exit_5_6 = pd.read_sql_query(query_5_6,connection_project)
display(exit_5_6) 

,count
0,3553


Разработкой ПО занимаются 3,5к организаций, довольно большое число организаций конкурирует в этой среде. Это может говорить об развитии рынка IT, поскольку многие компании и население стараются оптимизировать свою работу через компьютер. Соответственно будет повышаться спрос на специалистов IT, что приведет к увеличнию вакансий в этой области.

#### Для компании «Яндекс» вывести список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавить строку Total с общим количеством вакансий компании.

In [27]:
# Для получения списка городов миллиоников выгрузим данные с сайта Википедия 

# Импортируем библиотеку requests
import requests 
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
# получаем содержимое ссылки
response = requests.get(url)

# Импортируем библиотеку BeautifulSoup
from bs4 import BeautifulSoup
# Создаём объект BeautifulSoup, указывая параметр html-парсер
page = BeautifulSoup(response.text, 'html.parser') 


l = list()
# указываем тег и класс нужной таблицы 
table = page.select('table', class_='standard sortable jquery-tablesorter')

# В циклах двигаемся по html таблице и получаем названия городов 
for i in table:
    for td in i.select('td'):
        for a in td.select('a'):
            l.append(a.get_text('title'))
            
l.remove('[a]')
tuple_city = tuple(l[1:17]) # всего 16 городов миллиоников, поэтому указываем 16 значений
print(tuple_city)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [28]:
query_5_7 = f''' 
            SELECT 
                ar.name,
                COUNT(vc.id)
            
            FROM public.vacancies vc
                JOIN public.employers em ON em.id = vc.employer_id
                JOIN public.areas ar ON ar.id = vc.area_id
            WHERE em.name = 'Яндекс' AND ar.name in {tuple_city}
            GROUP BY 1
            
            UNION
            
            SELECT 
                'total', COUNT(*)
            FROM public.vacancies vc
                JOIN public.employers em ON em.id = vc.employer_id
                JOIN public.areas ar ON ar.id = vc.area_id
            WHERE em.name = 'Яндекс' AND ar.name in {tuple_city}
            ORDER BY 2
            '''

In [29]:
exit_5_7 = pd.read_sql_query(query_5_7,connection_project)
display(exit_5_7) 

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


Вакансии Яндекса представлены в каждом городе-миллионнике, организация имеет широкую известность и это способствует развитию бренда на разные товары или услуги. 

# Предметный анализ

#### Сколько вакансий имеет отношение к данным?

In [30]:
query_6_1 = f''' 
            SELECT 
                COUNT(name)
            FROM public.vacancies
            WHERE lower(name) LIKE '%данн%' OR
                lower(name) LIKE '%data%'
            '''

In [31]:
exit_5_7 = pd.read_sql_query(query_6_1, connection_project)
display(exit_5_7)

,count
0,1771


#### Сколько есть подходящих вакансий для начинающего дата-сайентиста?

In [32]:
query_6_2 = f''' 
            SELECT
                count(*)
            FROM public.vacancies
            WHERE 
            upper(name) NOT LIKE '%HTML%' AND
                (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') AND
                
                (lower(name) LIKE '%junior%' OR
                experience LIKE '%Нет опыта%' OR
                employment LIKE '%Стажировка%') 
            '''
            

In [33]:
exit_6_2 = pd.read_sql_query(query_6_2,connection_project)
display(exit_6_2)

,count
0,51


Среди всех вакансий около 50 подходят начинающим специалистам data science. Профессия новая и число вакансий не велико, но спрос будет расти, поскольку информационные технологии помогают организациям собирать огромные объёмы информации, которые необходимо обрабатывать и извлекать из них ценные данные. 
Для оценки положения среди всех вакансий связанных с data, проведем дополнительное исселедование

#### **Дополнительное исследование №3**

In [34]:
query_test4 = f''' 
            SELECT -- выведем вакансии для data science
                count(*)
            FROM public.vacancies
            WHERE 
            upper(name) NOT LIKE '%HTML%' AND
                (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
            '''
exit_test4 = pd.read_sql_query(query_test4,connection_project)

# Создадим новый dataframe для постройки графика 
data_test = exit_5_7.copy()
data_test.loc[1] = exit_test4.loc[0]
data_test.loc[2] = exit_6_2.loc[0]
data_test['level'] = pd.DataFrame({'level':['all_data','all_science','junior_science']})
display(data_test)


plot_test4 = px.pie(data_frame = data_test,
                    values= 'count',
                    names= 'level',
                    title= 'Распределение вакансий по специальности data'
                    )
plot_test4.show()
plot_test4.write_html("plotly_express4.html")

,count,level
0,1771,all_data
1,536,all_science
2,51,junior_science


Специальности, связанные с data science, занимают 1/4 часть от всех вакансий, связанных с большими данными.
Остальные вакансии так или иначе связаны с обработкой, анализом и хранением данных.

Можно сделать вывод, что большинство работодателей ищут в первую очередь специалиста, который сможет извлекать из данных нужную информацию, заниматься анализом и обработкой. 
Во вторую, это использование big data для машинного обучения. Такие специалисты, в основном, нужны крупным компаниям, посольку такого объёма данных у молодых компаний может не быть.

Далее видим, что junior-специалисты не сильно востребваны относительно более опытых кандидатов.
Это подтверждает выводы о том, что IT рынок только начал развиваться и большинству компаний нужны работнки, способные быстро начать приносить пользу для бизнеса. Начинающих же специалистов готовы брать в большинстве крупные компании, имеющие необходимые для обучения ресурсы.

#### Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [ ]:
query_6_3 = f''' 
            SELECT
                COUNT(*)
            FROM public.vacancies
            WHERE 
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') AND
                
                upper(name) NOT LIKE '%HTML%' AND
                lower(name) NOT LIKE '%teamlead%' AND
                (lower(key_skills) LIKE '%sql%' OR lower(key_skills) LIKE '%postgre%')
            '''

In [ ]:
exit_6_3 = pd.read_sql_query(query_6_3,connection_project)
display(exit_6_3)

,count
0,201


Навык работы с SQL требуется часто в работе саентиста, поскольку в этой профессии большую часть времени может занимать работа с данными - обработка, анализ, очистка. С помощью SQL можно быстро провести анаиз, получить нужную информацию и с помощью систем BI создать дашборд.

#### Вычислить количество вакансий, в которых в качестве ключевого навыка указан Python.

In [ ]:
query_6_4 = f'''
            SELECT
                COUNT(*)
            FROM public.vacancies
            WHERE lower(key_skills) LIKE '%python%' AND
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') AND
                
                upper(name) NOT LIKE '%HTML%' AND
                lower(name) NOT LIKE '%teamlead%' 
               '''

In [ ]:
exit_6_4 = pd.read_sql_query(query_6_4,connection_project)
display(exit_6_4)

,count
0,351


Знание языка python требуется больше чем в половине вакансий для саентистов. Это связано с тем, что он является основным для машинного обучения. Довольно простой и в использовании, гибкий, множество библиотек - все это делает python самым востребованым языком в создании искуственного интеллекта.

#### Сколько ключевых навыков в среднем указывают в вакансиях для DS?

In [11]:
query_6_5 = f''' 
            SELECT 
                AVG(
                    LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '\t','')) +1
                    )
            FROM public.vacancies
            WHERE
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' AND
                key_skills IS NOT NULL
            '''

In [12]:
exit_6_5 = pd.read_sql_query(query_6_5,connection_project)
display(exit_6_5)

,avg
0,6.406032


In [14]:
# ВТОРОЙ ВАРИАНТ ВЫПОЛНЕНИЯ ЗАДАНИЯ 6_5
query_6_5 = f''' 
            SELECT 
                key_skills
            FROM public.vacancies
            WHERE
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%'
                AND key_skills IS NOT NULL
            '''
exit_6_5 = pd.read_sql_query(query_6_5,connection_project)

# создаем объет dataframe
ser = pd.DataFrame(exit_6_5) 

# Выбираем значения без None, ищем в каждой записи значения '/t' и считаем навыки
def func(data):
    
    if data is None:
        return 0 # При возвращении 0, мы повлияем на среднее значение, поэтому в запросе мы ставим условие key_skills IS NOT NULL. 
    
    str_skill = list(data)
    sum_skill = str_skill.count('\t') 
    return sum_skill +1

series = ser['key_skills'].apply(func)
#создаём новый датафрайм с кол-вом навыков
new_data = pd.DataFrame(series)
display(round(new_data.mean(),2))


key_skills    6.41
dtype: float64

#### Какую зарплату для DS в среднем указывают для каждого типа требуемого опыта.



In [ ]:
query_6_6 = f''' 
            WITH with_null AS 
            (SELECT 
            -- Составим столбцец размера оплаты в случаи, если указана только одна граница зп вилки
                experience,
                coalesce(salary_from, 0) + coalesce(salary_to, 0) mean
                         
            FROM public.vacancies
            WHERE 
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' AND
                (salary_from IS NOT NULL OR
                salary_to IS NOT NULL) AND
                (salary_from + salary_to) IS NULL
                
            UNION ALL
            
            SELECT 
            -- Получим размер оплаты в случаи, если указаны обе границы зп вилки и
            -- объеденим с предыдущим запросом 
                experience,
                (salary_from + salary_to)/2
                
            FROM public.vacancies
            WHERE 
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' AND
                (salary_from IS NOT NULL OR
                salary_to IS NOT NULL) AND
                (salary_from + salary_to) IS NOT NULL
                )
                
            SELECT 
            -- Группируем по столбцу опыта и считаем среднее значение зарплат 
                experience,
                AVG(mean)
            FROM  with_null
            GROUP BY 1
            '''


In [ ]:
exit_6_6 = pd.read_sql_query(query_6_6,connection_project)
display(exit_6_6)

connection_project.close()

,experience,avg
0,От 3 до 6 лет,243114.666667
1,От 1 года до 3 лет,139674.750000
2,Нет опыта,74642.857143


# Общий вывод по проекту

Проанализировав данные работодателей и вакансий, вынесем несколько предположений касательно рынка труда. 
На основании статистических вычеслений по вакансиям, мы наблюдаем признаки развивающегося рынка. Молодые компании привлекают начинающих специалистов,самых востребованных среди других, для получения в будущем опытного специалиста, а уже устойчивые организации - для расширения бизнеса. Так же количество вакансий с типом графика "удаленная работа" гораздо ниже, чем с "полным днем", что может говорить о развитии рынка IT.

Основные работодатели являются монополистами на некоторые услуги. Например Яндекс владеет поисковиком, Сбер - банком, Ростелеком - связью и тп.
Компании-монополисты стараются обхватить дополнительные сферы бизнеса и создавать свой бренд для разных продуктов. Наличие монополий имеет свои последствия для рынка, например угнетение  малого бизнеса, уменьшение конкуренции, ухудшение качества продукции и тп.

